<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/encrypted_deep_learning_addition_subtraction_scalar_multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project - XIV
## Objective : Implementing secret sharing techniques for more arithmetic computations

In [1]:
pip install syft

  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch as th
import numpy as np
import random
import syft as sy

W0717 06:04:39.663895 140222978795392 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0717 06:04:39.689613 140222978795392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
base = 10
int_precision = 8
frac_precision = 8
Q = 293973345475167247070445277780365744413


In [0]:
precision = int_precision + frac_precision
assert(Q > base**precision)


In [0]:
def encode(rational):
  upscaled = int(rational * base ** int_precision)
  return upscaled % Q

In [0]:
def decode(field_element):
  upscaled = field_element if field_element <= Q/2 else field_element - Q
  return upscaled / base ** frac_precision

In [0]:
def encrypt(secret):
  first = random.randrange(Q)
  second = random.randrange(Q)
  third = (secret - first - second) % Q
  return [first, second, third]

In [0]:
def decrypt(sharing):
  return sum(sharing) % Q

### Implementing addition 

In [0]:
def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [14]:
x = encrypt(encode(7.5))
x

[266260249539904992688567422491459012987,
 56811506939295496652495849157632252308,
 264874934471134004799827283912390223531]

In [15]:
y = encrypt(encode(5.5))
y

[132044802323810252824993632366689881932,
 240508720941896147633263715140458891633,
 215393167684628093682633208054132715261]

In [16]:
z = add(x, y)
z

(104331706388547998443115777077783150506,
 3346882406024397215314286517725399528,
 186294756680594851412015214186157194379)

In [19]:
decode(decrypt(z))

13.0

In [21]:
26 % base

6

### Let's understand the important idea of modular arithmetic to understand the technique of secret sharing to perform different arithmetic operations.

Let's say we encrypt some numbers and got 15 and 13 , then we add those and subtract those and take modulus what do we get ?

In [23]:
(15 + 13) % 10

8

In [24]:
(15 - 13) % 10

2

We see that actually we add 5 and 3 instead of 15 and 13 , thus encrypting the actual numbers.

In [0]:
field = 23740629843760239486723


In [0]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [27]:
(bob_x_share + alices_x_share) % field


5

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [29]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field


4

In [30]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field


4

In [31]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share


26

In [0]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [33]:
(bob_z_share + alice_z_share) % field


4

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 6

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [40]:
z = bob_x_share + alice_x_share + bob_y_share + alice_y_share
z

31

In [41]:
z % base

1

So as we can see above , after we try to decrypt the values again , then we are not able to get back our original sum that is 11 but rather it is giving 1 and that is because we are overflowing and so we have to set field size consiciously.

In [0]:
field = 23740629843760239486723

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 6

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [46]:
z = bob_x_share + alice_x_share + bob_y_share + alice_y_share
z

47481259687520478973457

In [47]:
z % base

7

In [0]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [0]:
x = encrypt(20)
y = encrypt(10)

In [52]:
decrypt(sub(x, y))

10

In [53]:
((bob_y_share * 3) + (alice_y_share * 3)) % field


18

In [0]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [55]:
x = encrypt(encode(5.5))
x

[224009123369526855104583435363557068590,
 32775378292633214959099487506461480475,
 37188843813007177006762354910897195348]

In [0]:
z = imul(x, 3)


In [57]:
decode(decrypt(z))


16.5